In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./data/processed/final.csv")

In [3]:
data.head()

,question,answer,question_clean,answer_clean
0,Chỉ số xét nghiệm ca 125 là gì?,Xét nghiệm CA 125 chính là chỉ số kháng nguyên...,chỉ_số xét_nghiệm ca 125,xét_nghiệm ca 125 chỉ_số kháng_nguyên ung_thư ...
1,3 món ăn đơn giản giúp ngực to một cách tự nhiên,Có thể bạn không biết những món ăn đơn giản th...,3 món đơn_giản giúp ngực to tự_nhiên,món đơn_giản thường_ngày công_dụng ngực to căn...
2,Giá khám tổng quát định kì,Như các bạn biết thì chi phí khám sức khỏe tổn...,giá khám tổng_quát định_kì,chi_phí khám sức_khỏe tổng_quát khoản khám sức...
3,Đừng để bị giảm thị lực vĩnh viễn do viêm bờ mi,"Bạn có thể gặp nhiều phiền toái, mất thẩm mĩ v...",đừng thị_lực vĩnh_viễn viêm bờ mi,phiền_toái thẩm_mĩ mắt viêm bờ mi có_thể bị th...
4,5 thời điểm vàng bạn nên uống mật ong,Bạn dùng mật ong vào buổi sáng để lọc sạch cặn...,5 vàng uống mật_ong,mật_ong lọc sạch cặn bẩn đêm detox detox mật_o...


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

In [5]:
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [6]:
def embed(text):
    input_ids = torch.tensor([tokenizer.encode(text)])
    with torch.no_grad():
        features = phobert(input_ids)
    return features[1][0]

In [7]:
data[data['question_clean'].isnull()]

,question,answer,question_clean,answer_clean


In [8]:
import numpy as np
def consine(x,y):
    x = np.array(x)
    y = np.array(y)
    return x.dot(y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [9]:
from src.cleaner import Cleaner
clean = Cleaner()

In [10]:
# Đánh đổi bộ nhớ
data['question_embed'] = data['question_clean'].apply(lambda t: embed(t))

In [11]:
def get_similary(text, n):
    """ Select n question that similary with text """
    text = clean.run(text)
    input_embed = embed(text)
    result = {}
    for q, v in zip(data['question'], data['question_embed']):
        result[q] = consine(input_embed, v)
    return sorted(result.items(), key=lambda x: x[1], reverse=True)[:n]

In [12]:
%%time
get_similary("bệnh viêm gan b là gì?", 5)

CPU times: user 449 ms, sys: 4.02 ms, total: 453 ms
Wall time: 246 ms


[('Chữa viêm gan B mất bao nhiêu thời gian?', 0.9280161),
 ('Khi nào thì nên dừng điều trị viêm gan B?', 0.8710027),
 ('Xét nghiệm viêm gan B ở đâu là tốt nhất?', 0.8566841),
 ('Các bước xét nghiệm viêm gan B bạn cần biết', 0.8566841),
 ('Nước tiểu vàng có phải do viêm gan B gây nên?', 0.85390073)]

In [13]:
%%time
get_similary("làm gì khi bị ung thư?", 5)

CPU times: user 432 ms, sys: 12 µs, total: 432 ms
Wall time: 228 ms


[('Hiểu như thế nào là đúng về ung thư', 1.0),
 ("Những điều mà chỉ khi nào 'yêu' mới mang lại được", 0.613156),
 ('Tất cả những điều bạn cần biết về ung thư đại tràng', 0.58304715),
 ('Một số vấn đề thường gặp khi cho con bú và cách xử lý', 0.5827609),
 ('Cách điều trị ung thư tuyến giáp', 0.56907916)]

In [14]:
%%time
get_similary("tôi cảm thấy mình bị HIV? Tôi phải làm gì?", 5)


CPU times: user 428 ms, sys: 0 ns, total: 428 ms
Wall time: 221 ms


[('Bệnh viện FV có tốt không?', 0.7750233),
 ('Cách tính tuổi thai IVF như thế nào là đúng?', 0.7496771),
 ('Tại sao HIV không lây qua vết đốt của muỗi?', 0.72318),
 ('Những điều cần biết về viêm VA ở trẻ nhỏ', 0.7177166),
 ('Xét nghiệm HIV ở đâu chính xác nhất tại TP. Hồ Chí Minh?', 0.70913696)]

In [15]:
data['similary'] = data['question_clean']
data['score'] = data['question_clean']

In [16]:
# def find_similary(embed):
#     max_score = 0
#     best_similary = ''
#     for j in range(len(data)):
#         if data.iloc[j]['question_embed'] != embed:
#             q2 = data.iloc[j]['question_embed']
#             score = consine(q1, q2)
#             if score > max_score:
#                 max_score = score
#                 best_similary = data.iloc[j]['question']
#     return best_similary

In [17]:
# data['similary'] = data['question_embed'].apply(lambda e: find_similary(e))

In [20]:
import tqdm
for i in tqdm.tqdm(range(10)):
    q1 = data.iloc[i]['question_embed']
    max_score = 0
    best_similary = ''
    for j in range(len(data)):
        if j != i:
            q2 = data.iloc[j]['question_embed']
            score = consine(q1, q2)
            if score > max_score:
                max_score = score
                best_similary = data.iloc[j]['question']
    print(max_score, best_similary)
    data.iloc[i]['similary'] = best_similary
    data.iloc[i]['score'] = max_score

  0%|          | 0/10 [00:00<?, ?it/s]/home/thang/env/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/thang/env/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
100%|██████████| 10/10 [00:13<00:00,  1.33s/it]0.81100625 Những loại trà thảo mộc hữu ích cho bà bầu



In [21]:
data.head()

,question,answer,question_clean,answer_clean,question_embed,similary,score
0,Chỉ số xét nghiệm ca 125 là gì?,Xét nghiệm CA 125 chính là chỉ số kháng nguyên...,chỉ_số xét_nghiệm ca 125,xét_nghiệm ca 125 chỉ_số kháng_nguyên ung_thư ...,"[tensor(0.2440), tensor(-0.0822), tensor(0.023...",chỉ_số xét_nghiệm ca 125,chỉ_số xét_nghiệm ca 125
1,3 món ăn đơn giản giúp ngực to một cách tự nhiên,Có thể bạn không biết những món ăn đơn giản th...,3 món đơn_giản giúp ngực to tự_nhiên,món đơn_giản thường_ngày công_dụng ngực to căn...,"[tensor(0.2420), tensor(0.1790), tensor(-0.089...",3 món đơn_giản giúp ngực to tự_nhiên,3 món đơn_giản giúp ngực to tự_nhiên
2,Giá khám tổng quát định kì,Như các bạn biết thì chi phí khám sức khỏe tổn...,giá khám tổng_quát định_kì,chi_phí khám sức_khỏe tổng_quát khoản khám sức...,"[tensor(0.2270), tensor(0.1865), tensor(-0.131...",giá khám tổng_quát định_kì,giá khám tổng_quát định_kì
3,Đừng để bị giảm thị lực vĩnh viễn do viêm bờ mi,"Bạn có thể gặp nhiều phiền toái, mất thẩm mĩ v...",đừng thị_lực vĩnh_viễn viêm bờ mi,phiền_toái thẩm_mĩ mắt viêm bờ mi có_thể bị th...,"[tensor(0.1708), tensor(0.2054), tensor(-0.122...",đừng thị_lực vĩnh_viễn viêm bờ mi,đừng thị_lực vĩnh_viễn viêm bờ mi
4,5 thời điểm vàng bạn nên uống mật ong,Bạn dùng mật ong vào buổi sáng để lọc sạch cặn...,5 vàng uống mật_ong,mật_ong lọc sạch cặn bẩn đêm detox detox mật_o...,"[tensor(0.1663), tensor(0.1690), tensor(0.0535...",5 vàng uống mật_ong,5 vàng uống mật_ong
